### TODO

**Path Embedding**
- 一个可解释的，简洁的路径描述
- 增量式增加路径和节点的能力


**Link Prediction**
- 判断path头和path尾是否有连接

In [2]:
import pandas as pd
import os

In [9]:
path = 'subgraph'
src_list = os.listdir(path)
src_node = src_list[0]


nodes = pd.read_csv(os.path.join(path, src_node, f'sample_{src_node}.csv'))
edges = pd.read_csv(os.path.join(path, src_node, f'edge_{src_node}.csv'))
nodes

,pid,new_title,new_abs,label
0,340299,Title: Neural Network Applicability: Classifyi...,Abstract: tremendous effort propose neurally i...,Neural_Networks
1,1365,Title: Cholinergic suppression transmission co...,Abstract: Selective suppression transmission f...,Neural_Networks
2,34257,Title: Self-Adjusting Dynamic Logic Module,Abstract: ASOCS (Adaptive Self-Organizing Conc...,Neural_Networks
3,34263,Title: Word Perfect TRANSFORMATION IMPLEMENTIN...,Abstract: Artificial Neural Networks (ANNs) fi...,Neural_Networks
4,34266,"Title: VLSI Implementation Parallel, Self-Orga...",Abstract: VLSI implementation Priority Adaptiv...,Neural_Networks
5,1122642,Title: Word Perfect LIA: Location-Independent ...,Abstract: Artificial Neural Networks (ANNs) fi...,Neural_Networks
6,87482,Title: Multi-Chip Module Implementation Neural...,Abstract: requirement dense interconnect artif...,Neural_Networks
7,74427,Title: Self-Organizing Binary Decision Tree In...,Abstract: ASOCS (adaptive self-organizing conc...,Neural_Networks
8,45533,Title: Priority ASOCS ASOCS models two advanta...,Abstract: ASOCS (Adaptive Self-Organizing Conc...,Neural_Networks
9,213246,Title: Growing Layers Perceptrons: Introducing...,Abstract: vations perceptrons: (1) perceptron ...,Neural_Networks


![graph](subgraph/1122642/graph.png)

paper format:
- title:
- problems:
- methods:
- discovery:
- keywords

Evolution A to B: B builds on A by introducing a new technology, X, which delves into problem Y and presents viewpoint Z.

In [8]:
# 第一个思路，单纯做一条路径的文本表示，不去管其他邻居
# query path [1122642, 34257, 87482, 34263, 34266]

# 第二个思路，对于每个节点，嵌入他们社群子图的信息，然后做路径表示，然后做LP

In [13]:
# summary_prompt

for i in range(len(nodes)):
    pid, title, abstract, label = nodes.loc[i, 'pid'], nodes.loc[i, 'new_title'], nodes.loc[i, 'new_abs'], nodes.loc[i, 'label']

    prompt = f'''
You are an expert in summarising the information in a paper, now given the title, abstract and labels of the paper, you need to summarise the information in the paper and organise the content into the following format:
title:
problems:
methods:
discovery:
keywords:

paper information:
title: {title}
abstract: {abstract}
label: {label}
'''
    print(pid)
    print(prompt)
    print('==============================================================================================================')

340299

You are an expert in summarising the information in a paper, now given the title, abstract and labels of the paper, you need to summarise the information in the paper and organise the content into the following format:
title:
problems:
methods:
discovery:
keywords:

paper information:
title: Title: Neural Network Applicability: Classifying Space
abstract: Abstract: tremendous effort propose neurally inspired methods computation forces closer scrutiny potential models. categorizes applications classes discusses features applications efficiently amenable neural network methods. Computational machines deterministic mappings inputs outputs computational mechanisms solutions. Neural network features parallel execution, adaptive learning, generalization, fault tolerance. Often, effort model applications implemented efficient alternate technology. Neural networks powerful devices classes applications, all. However, class applications neural networks efficient commonly occurring nature

In [16]:
# transition prompt

nodes = pd.read_excel('subgraph/1122642/sample.xlsx')
edges = pd.read_csv('subgraph/1122642/edge_1122642.csv')

for i in range(len(edges)):
    new_paper, old_paper = edges.loc[i, 'cite_id'], edges.loc[i, 'cited_id']

    # old_paper information
    old_paper_info = nodes.loc[nodes.pid == old_paper, 'summary'].values[0]
    # new_paper information
    new_paper_info = nodes.loc[nodes.pid == new_paper, 'summary'].values[0]

    prompt = f'''
You are an expert in summarising the evolution of papers, and are given information about two papers A and B, and B cites A. You need to summarise them in the following format:
paper A title:
paper B title:
Evolution A to B: B builds on A by introducing a new technology, X, which delves into problem Y and presents viewpoint Z.

paper A information: 
{old_paper_info}
paper B information: 
{new_paper_info}
'''
    print(new_paper, old_paper)
    print(prompt)
    print('================================================================================================================')

1365 340299

You are an expert in summarising the evolution of papers, and are given information about two papers A and B, and B cites A. You need to summarise them in the following format:
paper A title:
paper B title:
Evolution A to B: B builds on A by introducing a new technology, X, which delves into problem Y and presents viewpoint Z.

paper A information: 
title: Neural Network Applicability: Classifying Space
problems:

Efficient classification of various applications using neural networks.
Comparison of neural network-based systems with supervised, unsupervised, and generalizing systems.
methods:
Utilization of neural network features such as parallel execution, adaptive learning, generalization, and fault tolerance.
Categorization of applications into classes and discussion of features amenable to neural network methods.
discovery:
Neural networks offer powerful computational capabilities for various applications.
Efficiency concerns arise in applying neural networks to common

In [18]:
# Link Prediction prompt

# query path [1122642, 34257, 87482, 34263, 34266]

# [Head paper, paper1, paper2, ..., Tail paper]

nodes = pd.read_excel('subgraph/1122642/sample.xlsx')
edges = pd.read_excel('subgraph/1122642/edge.xlsx')

# 确定头节点，尾节点
def chain_desc(head, tail, query):
    tail_idx = query.index(tail)

    head_desc, tail_desc = nodes.loc[nodes.pid == head, 'summary'].values[0], nodes.loc[nodes.pid == tail, 'summary'].values[0]
    prompt = ""
    

    if tail_idx == 1:
        evo = edges[(edges.cited_id == head) & (edges.cite_id == tail)]['evolution'].values[0]
        prompt += "head paper:" + head_desc
        prompt += "tail paper:" + tail_desc
        prompt += "evolution from head to tail:" + evo
        return  prompt
    
    else:
        prompt += "head paper:" + head_desc
        for i in range(1, tail_idx):
            node_desc = nodes.loc[nodes.pid == query[i], 'summary'].values[0]
            evo = edges[((edges.cited_id == query[i-1]) & (edges.cite_id == query[i])) | ((edges.cited_id == query[i]) & (edges.cite_id == query[i-1]))]['evolution'].values[0]
            prompt += f"paper {i}:" + node_desc 
            if i == 1:
                prompt += f"evolution from head to paper{i}:" + evo
            else:
                prompt += f"evolution from paper{i-1} to paper{i}:" + evo
        prompt += "tail paper:" + tail_desc
        evo = 
        


src_node = 1122642
path = [1122642, 34257, 87482, 34263, 34266]

print(chain_desc(src_node, 34257, path))

# for node in path:
#     if node == src_node:
#         continue
#     prompt = f'''
# You are an expert in determining whether a citation relationship exists, and now given information about a chain of paper citations, you need to determine whether the trailing paper cites the head paper:
# citation chain information:
# '''

head paper:title: Word Perfect LIA: Location-Independent Transformation ASOCS Adaptive Algorithm
problems:

Addressing the limitations of fixed-topology learning in Artificial Neural Networks (ANNs).
Overcoming these limitations by employing dynamic topologies in ANNs.
methods:
Introducing Location-Independent Transformations (LITs) as a strategy for efficiently implementing dynamic topologies in neural network learning models on parallel hardware.
Creating location-independent nodes through LITs, allowing nodes to compute network output using local information and support dynamic addition and deletion during learning.
Presenting the Location Independent ASOCS (LIA) model, which incorporates LITs into the ASOCS Adaptive Algorithm 2, and providing formal definitions and descriptions of LIA algorithms.
discovery:
LIA model enhances ASOCS mechanisms by employing LITs, enabling dynamic topology adjustments in neural network learning models.
The incorporation of LITs into ASOCS facilitates 